# DVC Options - Metrics and Plots
## The DVC way

In the following part we will look into metrics and plots from ZnTrack Nodes.
All `dvc run` options listed [here](https://dvc.org/doc/command-reference/run#options) can be used via `dvc.<option>`.
With the exception of params, which is handled automatically.
All these options take either `str` or `pathlib.Path` directed to the file the content should be stored in.
As shown before, `dvc.deps` can also take another `Node` as an argument.

In [1]:
from zntrack import Node, dvc, zn, config
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [2]:
config.nb_name = "04_metrics_and_plots.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpsbgdddyr/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In the following we define a simple Node that produces a metric and a plot output using `json` and `pandas`.
We will queue multiple experiments with different outputs and then compare them afterwards.
With `Node.write_graph(silent=True)` we can reduce the amount of logs that will be displayed.

In [5]:
class MetricAndPlot(Node):
    my_metric: Path = dvc.metrics(Path("my_metric.json"))
    my_plots: Path = dvc.plots("my_plots.csv")
    pre_factor = zn.params()

    def __init__(self, pre_factor=1.0, **kwargs):
        super().__init__(**kwargs)
        self.pre_factor = pre_factor

    def run(self):
        self.my_metric.write_text(
            json.dumps(
                {"metric_1": 17 * self.pre_factor, "metric_2": 42 * self.pre_factor}
            )
        )

        x_data = np.linspace(0, 1.0 * self.pre_factor, 1000)
        y_data = np.exp(x_data)
        df = pd.DataFrame({"y": y_data, "x": x_data}).set_index("x")

        df.to_csv(self.my_plots)

In [6]:
MetricAndPlot().write_graph(silent=True)
!dvc repro
!git add .
!git commit -m "First Run"

ore>Running stage 'MetricAndPlot':
> python3 -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot.load(name='MetricAndPlot').run_and_save()" 
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

    git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.
[master (root-commit) 092ee7a] First Run
 17 files changed, 1373 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/confusion_normalized.json
 create mode 100644 .dvc/plots/linear.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/simple.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 .dvcignore
 create mode 100644 .gitignore
 create mode 100644 04_metrics_and_plots.ipynb
 create mode 100644 dvc.lock

In [7]:
MetricAndPlot(pre_factor=2).write_graph(silent=True)
!dvc exp run --queue --name "factor_2"
MetricAndPlot(pre_factor=3).write_graph(silent=True)
!dvc exp run --queue --name "factor_3"
MetricAndPlot(pre_factor=4).write_graph(silent=True)
!dvc exp run --queue --name "factor_4"
MetricAndPlot(pre_factor=5).write_graph(silent=True)
!dvc exp run --queue --name "factor_5"

ore>Queued experiment 'b7c5df1' for future execution.
ore>Queued experiment '347519f' for future execution.
ore>Queued experiment '3a97e79' for future execution.
ore>Queued experiment '643eda6' for future execution.


In [8]:
!dvc exp run --run-all -j 4

  0% Checkout|                                       |0/2 [00:00<?,     ?file/s]
!
  0%|          |.jTFNbSYCnpKeGjzppQy8xm.tmp        0.00/? [00:00<?,        ?B/s]
  0%|          |.jTFNbSYCnpKeGjzppQy8xm.tmp     0.00/4.00 [00:00<?,        ?B/s]
  0%|          |.jTFNbSYCnpKeGjzppQy8xm.tmp     0.00/4.00 [00:00<?,        ?B/s]
                                                                                
!
  0%|          |4d8533a8bd0c3e8c73e09a05b5bf76     0.00/? [00:00<?,        ?B/s]
  0%|          |4d8533a8bd0c3e8c73e09a05b5bf76  0.00/36.0 [00:00<?,        ?B/s]
  0%|          |4d8533a8bd0c3e8c73e09a05b5bf76  0.00/36.0 [00:00<?,        ?B/s]
  0% Checkout|                                       |0/2 [00:00<?,     ?file/s]
!
  0%|          |.ciY4wongDaXs23fidDeBk3.tmp        0.00/? [00:00<?,        ?B/s]
  0%|          |.ciY4wongDaXs23fidDeBk3.tmp     0.00/4.00 [00:00<?,        ?B/s]
  0%|          |.ciY4wongDaXs23fidDeBk3.tmp     0.00/4.00 [00:00<?,        ?B/s]
                      

Now that all experiments are done, we can look at the metrics directly with `dvc exp show` or `dvc metrics show/diff`

In [9]:
!dvc exp show --csv --no-timestamp > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

ore>

,rev,typ,parent,metric_1,metric_2,MetricAndPlot.pre_factor
Experiment,,,,,,
NaN,workspace,baseline,NaN,17.0,42.0,5.0
master,092ee7a,baseline,NaN,17.0,42.0,1.0
factor_4,b8c5e94,branch_commit,NaN,68.0,168.0,4.0
factor_2,76f8e0c,branch_commit,NaN,34.0,84.0,2.0
factor_3,13b720a,branch_commit,NaN,51.0,126.0,3.0
factor_5,8ac42fa,branch_base,NaN,85.0,210.0,5.0


We can also use `dvc plots show/diff` to evaluate the plot data that we produced.

In [10]:
!dvc plots diff HEAD factor_2 factor_3 factor_4 factor_5

file:///tmp/tmpsbgdddyr/dvc_plots/index.html


## The ZnTrack way

ZnTrack provides and easier way to handle metrics. Similar to `zn.outs()` which does not require defining a path to outs file, one can use `zn.metrics`.
The same is possible for plots via `zn.plots()` which requires a `pd.DataFrame` with a defined index name.

In [11]:
class ZnTrackMetric(Node):
    my_metric = zn.metrics()
    my_plot = zn.plots()

    def run(self):
        self.my_metric = {"alpha": 1.0, "beta": 0.00473}
        self.my_plot = pd.DataFrame({"val": np.sin(np.linspace(0, 3.14, 100))})
        self.my_plot.index.name = (  # For DVC it is required that the index has a column name
            "index"
        )


ZnTrackMetric().write_graph(no_exec=False)

2022-01-21 11:42:10,470 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-21 11:42:10,472 (WARNING): Converting 04_metrics_and_plots.ipynb to file ZnTrackMetric.py


[NbConvertApp] Converting notebook 04_metrics_and_plots.ipynb to script


2022-01-21 11:42:13,557 (WARNING): --- Writing new DVC file! ---
2022-01-21 11:42:13,558 (WARNING): You will not be able to see the stdout/stderr of the process in real time!


[NbConvertApp] Writing 3952 bytes to 04_metrics_and_plots.py


2022-01-21 11:42:16,642 (INFO): Running stage 'ZnTrackMetric':
> python3 -c "from src.ZnTrackMetric import ZnTrackMetric; ZnTrackMetric.load(name='ZnTrackMetric').run_and_save()" 
Adding stage 'ZnTrackMetric' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

    git add dvc.lock dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true



In [12]:
!dvc exp show --csv --no-timestamp > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

ore>

,rev,typ,parent,metric_1,metric_2,my_metric.alpha,my_metric.beta,MetricAndPlot.pre_factor
Experiment,,,,,,,,
NaN,workspace,baseline,NaN,17.0,42.0,1.0,0.00473,5.0
master,092ee7a,baseline,NaN,17.0,42.0,1.0,NaN,NaN
factor_4,b8c5e94,branch_commit,NaN,68.0,168.0,4.0,NaN,NaN
factor_5,8ac42fa,branch_commit,NaN,85.0,210.0,5.0,NaN,NaN
factor_2,76f8e0c,branch_commit,NaN,34.0,84.0,2.0,NaN,NaN
factor_3,13b720a,branch_base,NaN,51.0,126.0,3.0,NaN,NaN


In [13]:
!dvc plots show

file:///tmp/tmpsbgdddyr/dvc_plots/index.html


In [14]:
temp_dir.cleanup()